In [6]:
import pygame
import random
import sys

pygame.init()

WIDTH = 450
HEIGHT = 600
BLOCK_SIZE = 30
GRID_WIDTH = 10
GRID_HEIGHT = 20
FPS = 60

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
CYAN = (0, 255, 255)
YELLOW = (255, 255, 0)
MAGENTA = (255, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)

COLORS = [CYAN, YELLOW, MAGENTA, RED, GREEN, BLUE, ORANGE]

SHAPES = [
    [[1, 1, 1, 1]],
    [[1, 1], [1, 1]],
    [[1, 1, 1], [0, 1, 0]],
    [[1, 1, 1], [1, 0, 0]],
    [[1, 1, 1], [0, 0, 1]],
    [[1, 1, 0], [0, 1, 1]],
    [[0, 1, 1], [1, 1, 0]]
]

class Tetromino:
    def __init__(self):
        self.shape = random.choice(SHAPES)
        self.color = random.choice(COLORS)
        self.x = GRID_WIDTH // 2 - len(self.shape[0]) // 2
        self.y = 0

    def rotate(self):
        self.shape = [list(row) for row in zip(*self.shape[::-1])]

    def move(self, dx, dy):
        self.x += dx
        self.y += dy

class Tetris:
    def __init__(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Tetris")
        self.clock = pygame.time.Clock()
        self.reset_game()

    def reset_game(self):
        self.grid = [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        self.current_piece = Tetromino()
        self.game_over = False
        self.drop_time = 0
        self.base_drop_speed = 50
        self.drop_speed = self.base_drop_speed
        self.score = 0
        self.last_speed_update = 0
        self.highscore = max(getattr(self, 'highscore', 0), self.score)

    def update_drop_speed(self):
        current_threshold = self.score // 1000
        if current_threshold > self.last_speed_update:
            self.drop_speed = max(self.base_drop_speed * (1 / 1.5) ** current_threshold, 10)
            self.last_speed_update = current_threshold

    def check_collision(self, piece, x, y):
        for i in range(len(piece.shape)):
            for j in range(len(piece.shape[0])):
                if piece.shape[i][j]:
                    new_x = x + j
                    new_y = y + i
                    if (new_x < 0 or new_x >= GRID_WIDTH or new_y >= GRID_HEIGHT or
                        (new_y >= 0 and self.grid[new_y][new_x])):
                        return True
        return False

    def merge_piece(self):
        for i in range(len(self.current_piece.shape)):
            for j in range(len(self.current_piece.shape[0])):
                if self.current_piece.shape[i][j]:
                    self.grid[self.current_piece.y + i][self.current_piece.x + j] = self.current_piece.color

    def clear_lines(self):
        lines_cleared = 0
        new_grid = [row for row in self.grid if not all(row)]
        lines_cleared = GRID_HEIGHT - len(new_grid)
        for _ in range(lines_cleared):
            new_grid.insert(0, [0 for _ in range(GRID_WIDTH)])
        self.grid = new_grid
        return lines_cleared

    def instant_drop(self):
        while not self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
            self.current_piece.move(0, 1)
        self.current_piece.move(0, -1)
        self.merge_piece()
        lines_cleared = self.clear_lines()
        self.score += lines_cleared * 100
        self.update_drop_speed()
        if self.score > self.highscore:
            self.highscore = self.score
        self.current_piece = Tetromino()
        if self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
            self.game_over = True

    def draw_button(self, text, x, y, width, height, color, text_color):
        pygame.draw.rect(self.screen, color, (x, y, width, height))
        font = pygame.font.Font(None, 36)
        text_surf = font.render(text, True, text_color)
        text_rect = text_surf.get_rect(center=(x + width / 2, y + height / 2))
        self.screen.blit(text_surf, text_rect)
        return pygame.Rect(x, y, width, height)

    def run(self):
        running = True
        while running:
            if not self.game_over:
                self.drop_time += 1
                if self.drop_time >= self.drop_speed:
                    self.drop_time = 0
                    self.current_piece.move(0, 1)
                    if self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
                        self.current_piece.move(0, -1)
                        self.merge_piece()
                        lines_cleared = self.clear_lines()
                        self.score += lines_cleared * 100
                        self.update_drop_speed()
                        if self.score > self.highscore:
                            self.highscore = self.score
                        self.current_piece = Tetromino()
                        if self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
                            self.game_over = True

                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_LEFT:
                            self.current_piece.move(-1, 0)
                            if self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
                                self.current_piece.move(1, 0)
                        if event.key == pygame.K_RIGHT:
                            self.current_piece.move(1, 0)
                            if self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
                                self.current_piece.move(-1, 0)
                        if event.key == pygame.K_DOWN:
                            self.current_piece.move(0, 1)
                            if self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
                                self.current_piece.move(0, -1)
                                self.merge_piece()
                                lines_cleared = self.clear_lines()
                                self.score += lines_cleared * 100
                                self.update_drop_speed()
                                if self.score > self.highscore:
                                    self.highscore = self.score
                                self.current_piece = Tetromino()
                        if event.key == pygame.K_UP:
                            self.current_piece.rotate()
                            if self.check_collision(self.current_piece, self.current_piece.x, self.current_piece.y):
                                for _ in range(3):
                                    self.current_piece.rotate()
                        if event.key == pygame.K_SPACE:
                            self.instant_drop()

                self.screen.fill(BLACK)
                for i in range(GRID_HEIGHT):
                    for j in range(GRID_WIDTH):
                        if self.grid[i][j]:
                            pygame.draw.rect(self.screen, self.grid[i][j],
                                             (j * BLOCK_SIZE, i * BLOCK_SIZE, BLOCK_SIZE - 1, BLOCK_SIZE - 1))
                for i in range(len(self.current_piece.shape)):
                    for j in range(len(self.current_piece.shape[0])):
                        if self.current_piece.shape[i][j]:
                            pygame.draw.rect(self.screen, self.current_piece.color,
                                             ((self.current_piece.x + j) * BLOCK_SIZE,
                                              (self.current_piece.y + i) * BLOCK_SIZE,
                                              BLOCK_SIZE - 1, BLOCK_SIZE - 1))
                font = pygame.font.Font(None, 36)
                score_text = font.render(f"Score: {self.score}", True, WHITE)
                highscore_text = font.render(f"Highscore: {self.highscore}", True, WHITE)
                self.screen.blit(score_text, (10, 10))
                self.screen.blit(highscore_text, (10, 50))
            else:
                self.screen.fill(BLACK)
                font = pygame.font.Font(None, 48)
                game_over_text = font.render("Game Over", True, WHITE)
                self.screen.blit(game_over_text, (WIDTH // 2 - game_over_text.get_width() // 2, HEIGHT // 2 - 100))
                score_text = font.render(f"Score: {self.score}", True, WHITE)
                self.screen.blit(score_text, (WIDTH // 2 - score_text.get_width() // 2, HEIGHT // 2 - 50))
                highscore_text = font.render(f"Highscore: {self.highscore}", True, WHITE)
                self.screen.blit(highscore_text, (WIDTH // 2 - highscore_text.get_width() // 2, HEIGHT // 2))
                restart_button = self.draw_button("Restart", WIDTH // 2 - 50, HEIGHT // 2 + 50, 100, 50, BLUE, WHITE)

                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                    
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        if restart_button.collidepoint(event.pos):
                            self.reset_game()

            pygame.display.flip()
            self.clock.tick(FPS)

        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = Tetris()
    game.run()


SystemExit: 